# Interactive maps with folium

- home: [Folium](https://python-visualization.github.io/folium/latest/index.html)
- based on [Leaflet](https://leafletjs.com/)
- pretty cool library for creating interactive html/java-script maps from python

## Installation

```bash
pip install folium
```

## Basics

In [ ]:
import folium
[x for x in dir(folium) if not x.startswith('_')]

In [ ]:
# A map of the London Area (initially)
latlon = (51.5, 0.0)
m = folium.Map(location=latlon)

# show coordingates
#folium.LatLngPopup().add_to(m)

# a Marker
#folium.Marker(location=(51.3, -0.4), popup='just an ordinary marker').add_to(m)

# a Circle with size in meters
#folium.Circle(location=(51.7, -0.4), radius=2000, color='red').add_to(m)

# also a Circle but size in pixels
#folium.CircleMarker(location=(51.7, +0.4), radius=20, color='green').add_to(m)

# a Polygon
#folium.Polygon(locations=[(51.31, 0.42),(51.31, 0.39),(51.29, 0.38),(51.29, 0.41)], fillColor='#3186cc', fillOpacity=0.4).add_to(m)

# change the Tiles
#folium.TileLayer('stamenterrain').add_to(m)

# switch between Tiles
#folium.LayerControl().add_to(m)

m

## Plotting Climada Data

In [ ]:
from climada.util.api_client import Client
c = Client()

uk_lp_30 = c.get_litpop(country='GBR', exponents=(3,0))
uk_lp_01 = c.get_litpop(country='GBR', exponents=(0,1))
uk_lp_11 = c.get_litpop(country='GBR', exponents=(1,1))

uk_lp_11.gdf.head()

In [ ]:
folium.GeoJson?

In [ ]:
latlon = (51.5, 0.0)
m = folium.Map(location=latlon)
maxv = uk_lp_30.gdf.value.max()
folium.GeoJson(
    data=uk_lp_30.gdf[
               uk_lp_30.gdf.latitude.between(51.3, 51.7)
             & uk_lp_30.gdf.longitude.between(-0.4, 0.4)
    ].loc[:,['geometry', 'value']],
    #marker=folium.Circle(radius=1500, fillColor='blue'),
    #style_function=lambda lp: {
    #    'opacity': 0,
    #    'fillColor': 'blue',
    #    'fillOpacity': 0.6 * lp['properties']['value']/maxv,
    #},
    #popup=folium.GeoJsonPopup(fields=['value']),
).add_to(m)
m

In [ ]:
from shapely import Polygon
latlon = (51.5, 0.0)
m = folium.Map(location=latlon)
maxv = uk_lp_30.gdf.value.max()

def raster_polygon(lat, lon, arc_res):
    return Polygon([
        (lon+arc_res/2, lat+arc_res/2),
        (lon+arc_res/2, lat-arc_res/2),
        (lon-arc_res/2, lat-arc_res/2),
        (lon-arc_res/2, lat+arc_res/2)])
        
data = uk_lp_30.gdf[
               uk_lp_30.gdf.latitude.between(51.3, 51.7)
             & uk_lp_30.gdf.longitude.between(-0.4, 0.4)
    ]

data['geometry'] = data.apply(lambda x: raster_polygon(x.latitude, x.longitude, 150/3600), axis=1)
folium.GeoJson(
    data=data,
    popup=folium.GeoJsonPopup(fields=['value']),
    style_function=lambda lp: {
        'opacity': 0,
        'fillColor': 'blue',
        'fillOpacity':0.6 * lp['properties']['value']/maxv,
    }
).add_to(m)
m

## Annotating large areas

`folium.Choropleth` is better suited for large areas then `folium.GeoJson` and provides a sophisticated interface for plotting choropleth maps.

In [ ]:
folium.Choropleth?

In [ ]:
from climada_folium import exposures_folium, raster_polygons

latlon = (51.5, 0.0)
m = folium.Map(location=latlon)

exposures_folium(
    exposures=uk_lp_30,
    polygon_maker=raster_polygons(150/3600),
    add_to_map=m
)
m

In [ ]:
exposures_folium(
    exposures=uk_lp_01,
    polygon_maker=raster_polygons(150/3600),
    fill_color='YlGn',
    add_to_map=m
)

exposures_folium(
    exposures=uk_lp_11,
    polygon_maker=raster_polygons(150/3600),
    fill_color='OrRd',
    add_to_map=m
)

folium.LayerControl().add_to(m)

m